In [29]:
import json
import numpy as np


In [37]:
labels_dataset = "./data/labels/1 label dataset.json"
values_dataset = "./data/values/1 values dataset.json"

In [38]:
landmark_list = json.load(open(values_dataset))
timestamps = [float(x) for x in list(landmark_list.keys())]

#loading labels from the key_log.txt file with format: timestamp,key
labels = json.load(open(labels_dataset))

In [39]:
len(labels), len(timestamps), len(landmark_list)

(316, 7298, 7298)

In [40]:
values = []
for val in landmark_list.values():
    values.append(list(val.values()))

values = np.array(values)
values.shape

(7298, 21, 3)

In [41]:
data = []
window_size = 14
overlap = 14
for i in range(0, len(values) - window_size, overlap):
    data.append(values[i:i+window_size])

data = np.array(data)
data.shape

(521, 14, 21, 3)

# Training autoencoder for self-supervised learning (easy way)
### This helps learning representation of dynamic gestures

In [42]:
import tensorflow as tf
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_shape = Input(shape=(window_size, 21, 3))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_shape)
x = MaxPooling2D((2, 3), padding='same')(x)
encoded = Conv2D(32, (3, 3), activation='relu', padding='same', name = "encoder")(x)
encoder_model = Model(input_shape, encoded)

decoder_input = Input(shape = (encoder_model.output_shape[1:]))
#x = Conv2D(32, (3, 3), activation='relu', padding='same')(decoder_input)
x = UpSampling2D((2, 3))(decoder_input)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((1, 1))(x)
decoded = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
decoder_model = Model(decoder_input, decoded)

encoded_representation = encoder_model(input_shape)
decoded_output = decoder_model(encoded_representation)
autoencoder = Model(inputs=input_shape, outputs=decoded_output)

autoencoder.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 14, 21, 3)]       0         
                                                                 
 model_6 (Functional)        (None, 7, 7, 32)          5088      
                                                                 
 model_7 (Functional)        (None, 14, 21, 3)         5059      
                                                                 
Total params: 10147 (39.64 KB)
Trainable params: 10147 (39.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(data, data, epochs=30, batch_size=32, shuffle=True, validation_split=0.1)

Epoch 1/30
 1/15 [=>............................] - ETA: 5s - loss: 0.2169

2023-07-11 20:29:46.113078: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 1s 62ms/step - loss: 0.0945 - val_loss: 0.0301
Epoch 2/30
10/15 [===================>..........] - ETA: 0s - loss: 0.0360

2023-07-11 20:29:47.089745: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 0s 15ms/step - loss: 0.0338 - val_loss: 0.0194
Epoch 3/30
15/15 [==============================] - 0s 15ms/step - loss: 0.0224 - val_loss: 0.0135
Epoch 4/30
15/15 [==============================] - 0s 15ms/step - loss: 0.0156 - val_loss: 0.0098
Epoch 5/30
15/15 [==============================] - 0s 15ms/step - loss: 0.0119 - val_loss: 0.0077
Epoch 6/30
15/15 [==============================] - 0s 14ms/step - loss: 0.0095 - val_loss: 0.0067
Epoch 7/30
15/15 [==============================] - 0s 14ms/step - loss: 0.0081 - val_loss: 0.0061
Epoch 8/30
15/15 [==============================] - 0s 14ms/step - loss: 0.0071 - val_loss: 0.0055
Epoch 9/30
15/15 [==============================] - 0s 14ms/step - loss: 0.0065 - val_loss: 0.0051
Epoch 10/30
15/15 [==============================] - 0s 14ms/step - loss: 0.0061 - val_loss: 0.0048
Epoch 11/30
15/15 [==============================] - 0s 14ms/step - loss: 0.0057 - val_loss: 0.0045
Epoch 12/30
15/15 [

# Finetuning the encoder part

In [44]:
#fine tuning the encoder model
from keras.models import Sequential
from keras.layers import Flatten

#freeze the encoder layers
for layer in encoder_model.layers:
    layer.trainable = False

additional_layers = Sequential([
    Input(shape = (encoder_model.output_shape[1:])),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(4, activation='sigmoid')
]
)

In [45]:
model = Sequential([encoder_model, additional_layers])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_6 (Functional)        (None, 7, 7, 32)          5088      
                                                                 
 sequential_2 (Sequential)   (None, 4)                 205092    
                                                                 
Total params: 210180 (821.02 KB)
Trainable params: 205092 (801.14 KB)
Non-trainable params: 5088 (19.88 KB)
_________________________________________________________________


In [50]:
labels = json.load(open(labels_dataset, "r"))

#Now, from landmark_list, we want to extract a window of 0.5 seconds before and after each key press
#We will use the labels dictionary to label each window
#We will use the timestamps list to find the closest timestamp to the key press timestamp
#We will use the freq variable to calculate the number of frames we need to extract for each window
#We will use the loaded_landmark_list to extract the frames
window_size = 0.5  #0.5 seconds before and after the key press
freq = 14 #14 frames per second
window_frames = int(freq * window_size)
X, Y = [], []
for key in labels.keys():
    key = float(key)
    closest_timestamp = min(timestamps, key=lambda x:abs(x-key))
    index = timestamps.index(closest_timestamp)
    if index - window_frames < 0:
        continue
    if index + window_frames >= len(timestamps):
        continue
    
    window = []
    for i in range(index-window_frames, index+window_frames):
        k = str(timestamps[i])
        window.append(landmark_list[k])

    X.append(window)
    Y.append(labels[str(key)]['label'])

#retrive a structure with shape (n_windows, window_size, 21, 3)
data2 = []
for window in X:
    temp = []
    for sample in window:
        temp.append(np.array(list(sample.values())))
    
    data2.append(np.array(temp))

data = np.array(data2)
del data2


In [52]:
Y

['Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.left',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key.right',
 'Key

In [53]:
from sklearn.preprocessing import OneHotEncoder
#Y = [1 if y == 'Key.left' else 0 for y in Y]
Y = np.array(Y)
encoder = OneHotEncoder()
Y_encoded = encoder.fit_transform(Y.reshape(-1, 1)).toarray()

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, Y_encoded, test_size=0.2, random_state=42)

In [55]:
model.fit(X_train, y_train, epochs=30, batch_size=32, shuffle=True, validation_split=0.1)

Epoch 1/30
6/8 [=====================>........] - ETA: 0s - loss: 1.3096 - accuracy: 0.3333

2023-07-11 20:31:04.003463: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 50ms/step - loss: 1.3468 - accuracy: 0.3319 - val_loss: 1.0244 - val_accuracy: 0.4231
Epoch 2/30
8/8 [==============================] - 0s 15ms/step - loss: 1.0842 - accuracy: 0.4779 - val_loss: 0.6863 - val_accuracy: 0.8077
Epoch 3/30
1/8 [==>...........................] - ETA: 0s - loss: 0.7616 - accuracy: 0.7188

2023-07-11 20:31:04.421155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 19ms/step - loss: 0.7192 - accuracy: 0.8274 - val_loss: 0.5407 - val_accuracy: 0.8846
Epoch 4/30
8/8 [==============================] - 0s 16ms/step - loss: 0.5455 - accuracy: 0.8673 - val_loss: 0.4157 - val_accuracy: 0.8846
Epoch 5/30
8/8 [==============================] - 0s 15ms/step - loss: 0.4150 - accuracy: 0.9735 - val_loss: 0.2922 - val_accuracy: 0.9615
Epoch 6/30
8/8 [==============================] - 0s 16ms/step - loss: 0.3116 - accuracy: 0.9602 - val_loss: 0.2203 - val_accuracy: 0.9615
Epoch 7/30
8/8 [==============================] - 0s 18ms/step - loss: 0.2530 - accuracy: 0.9823 - val_loss: 0.1809 - val_accuracy: 0.9615
Epoch 8/30
8/8 [==============================] - 0s 16ms/step - loss: 0.1996 - accuracy: 0.9690 - val_loss: 0.1488 - val_accuracy: 0.9615
Epoch 9/30
8/8 [==============================] - 0s 16ms/step - loss: 0.1683 - accuracy: 0.9823 - val_loss: 0.1315 - val_accuracy: 0.9615
Epoch 10/30
8/8 [=====================

In [56]:
y_pred = model.predict(X_test)

2/2 [==============================] - 0s 21ms/step


2023-07-11 20:31:15.254837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [57]:
y_test.shape, y_pred.shape

((64, 4), (64, 4))

In [58]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

array([[ 8,  0,  0,  1],
       [ 0, 15,  0,  0],
       [ 0,  0, 31,  0],
       [ 0,  0,  1,  8]])

In [59]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test.argmax(axis = 1), y_pred.argmax(axis=1), average='weighted')
recall = recall_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
f1 = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
print("Precision: ", round(precision, 2))
print("Recall: ", round(recall, 2))
print("F1: ", round(f1, 2))

Precision:  0.97
Recall:  0.97
F1:  0.97


# Trying with baseline model

In [98]:
import keras
from keras.layers import Conv3D, MaxPooling2D, Flatten, Reshape, LSTM, Dense, Conv2D, Concatenate
num_classes = set(Y).__len__()
inp_shape = data.shape[1:]

# Input shape: (n_samples, 14, 21, 3)
input = Input(shape=inp_shape)
x = Conv2D(8, (2, 2), activation='relu', padding='same')(input)
x = MaxPooling2D((2, 3), padding='same')(x)
x = Conv2D(16, (2, 2), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 3), padding='same')(x)

y = Reshape((inp_shape[0]*inp_shape[1], inp_shape[2]))(input)
y = LSTM(8, return_sequences=True)(y)
y = Flatten()(y)
y = Dense(192, activation='relu')(y)
y = Reshape((4, 3, 16))(y)
y = Concatenate()([x, y])
y = Flatten()(y)
y = Dense(64, activation='relu')(y)
y = Dense(16, activation='relu')(y)
y = Dense(4, activation='softmax')(y)
model = Model(input, y)
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 14, 21, 3)]          0         []                            
                                                                                                  
 reshape_10 (Reshape)        (None, 294, 3)               0         ['input_20[0][0]']            
                                                                                                  
 conv2d_16 (Conv2D)          (None, 14, 21, 8)            104       ['input_20[0][0]']            
                                                                                                  
 lstm_5 (LSTM)               (None, 294, 8)               384       ['reshape_10[0][0]']          
                                                                                           

In [99]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, shuffle=True)

Epoch 1/30


2023-06-20 18:21:22.046176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 18:21:22.238436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 18:21:22.709695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - ETA: 0s - loss: 1.0268 - accuracy: 0.4954

2023-06-20 18:21:24.054583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 18:21:24.131479: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 3s 192ms/step - loss: 1.0268 - accuracy: 0.4954 - val_loss: 0.8257 - val_accuracy: 0.4167
Epoch 2/30
7/7 [==============================] - 0s 56ms/step - loss: 0.6081 - accuracy: 0.6991 - val_loss: 0.3790 - val_accuracy: 0.8750
Epoch 3/30
7/7 [==============================] - 0s 57ms/step - loss: 0.3907 - accuracy: 0.8519 - val_loss: 0.3642 - val_accuracy: 0.8750
Epoch 4/30
7/7 [==============================] - 0s 54ms/step - loss: 0.3128 - accuracy: 0.8981 - val_loss: 0.2569 - val_accuracy: 0.9583
Epoch 5/30
7/7 [==============================] - 0s 48ms/step - loss: 0.2709 - accuracy: 0.9352 - val_loss: 0.1751 - val_accuracy: 0.9583
Epoch 6/30
7/7 [==============================] - 0s 54ms/step - loss: 0.2275 - accuracy: 0.9398 - val_loss: 0.2705 - val_accuracy: 0.9583
Epoch 7/30
7/7 [==============================] - 0s 54ms/step - loss: 0.1718 - accuracy: 0.9769 - val_loss: 0.1184 - val_accuracy: 0.9583
Epoch 8/30
7/7 [=====================

In [100]:
y_pred = model.predict(X_test)

1/2 [==============>...............] - ETA: 0s

2023-06-20 18:21:37.104421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 18:21:37.164075: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 51ms/step


In [101]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

array([[ 8,  0,  0,  0],
       [ 0, 18,  0,  0],
       [ 0,  0, 27,  0],
       [ 0,  0,  0,  8]])

In [102]:
precision = precision_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
recall = recall_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
f1 = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
print("Precision: ", round(precision, 2))
print("Recall: ", round(recall, 2))
print("F1: ", round(f1, 2))

Precision:  1.0
Recall:  1.0
F1:  1.0


# just linear layers

In [103]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 14, 21, 3)]          0         []                            
                                                                                                  
 reshape_10 (Reshape)        (None, 294, 3)               0         ['input_20[0][0]']            
                                                                                                  
 conv2d_16 (Conv2D)          (None, 14, 21, 8)            104       ['input_20[0][0]']            
                                                                                                  
 lstm_5 (LSTM)               (None, 294, 8)               384       ['reshape_10[0][0]']          
                                                                                           

In [104]:
input = Input(shape=inp_shape)
x = Flatten()(input)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(4, activation='softmax')(x)
model = Model(input, x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, shuffle=True)

Epoch 1/30
1/7 [===>..........................] - ETA: 2s - loss: 2.0233 - accuracy: 0.1250

2023-06-20 18:21:47.523713: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 1s 51ms/step - loss: 1.9415 - accuracy: 0.5787 - val_loss: 1.5620 - val_accuracy: 0.7917
Epoch 2/30
4/7 [================>.............] - ETA: 0s - loss: 1.5971 - accuracy: 0.7891

2023-06-20 18:21:47.940392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 0s 22ms/step - loss: 1.3369 - accuracy: 0.7870 - val_loss: 0.7530 - val_accuracy: 0.7917
Epoch 3/30
7/7 [==============================] - 0s 20ms/step - loss: 0.4677 - accuracy: 0.8565 - val_loss: 0.3172 - val_accuracy: 0.9583
Epoch 4/30
7/7 [==============================] - 0s 19ms/step - loss: 0.2856 - accuracy: 0.9583 - val_loss: 0.2164 - val_accuracy: 0.9583
Epoch 5/30
7/7 [==============================] - 0s 19ms/step - loss: 0.2865 - accuracy: 0.9352 - val_loss: 0.1734 - val_accuracy: 0.9583
Epoch 6/30
7/7 [==============================] - 0s 18ms/step - loss: 0.1902 - accuracy: 0.9769 - val_loss: 0.1803 - val_accuracy: 0.9583
Epoch 7/30
7/7 [==============================] - 0s 18ms/step - loss: 0.1706 - accuracy: 0.9815 - val_loss: 0.1671 - val_accuracy: 0.9583
Epoch 8/30
7/7 [==============================] - 0s 19ms/step - loss: 0.1680 - accuracy: 0.9815 - val_loss: 0.1494 - val_accuracy: 0.9583
Epoch 9/30
7/7 [======================

In [105]:
y_pred = model.predict(X_test)
confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

2/2 [==============================] - 0s 7ms/step


2023-06-20 18:21:52.614759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


array([[ 8,  0,  0,  0],
       [ 0, 18,  0,  0],
       [ 0,  0, 27,  0],
       [ 0,  0,  0,  8]])

In [106]:
precision = precision_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
recall = recall_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
f1 = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
print("Precision: ", round(precision, 2))
print("Recall: ", round(recall, 2))
print("F1: ", round(f1, 2))

Precision:  1.0
Recall:  1.0
F1:  1.0
